In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch

In [ ]:
from datasets import load_dataset, Dataset

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model, AutoPeftModelForCausalLM

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, GPTQConfig

In [ ]:
from trl import SFTTrainer

In [ ]:
import os

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def generate_train():
    data = load_dataset("ananyarn/Algorithm_and_Python_Source_Code", streaming=True, split="train", download_mode="force_redownload")
    samp_total = 10000
    valid_p = 0.1
    samp_train=int(samp_total*(1-valid_p))
    count = 0
    for sample in iter(data):
        if count >= samp_train:
            break
        q = sample['Algorithm']
        start = q.find("1")
        question = q[start:].strip()
        answer = sample['output'].strip()
        nf = f'<s>[INST]{question}[/INST] ```python\n{answer}```</s>'
        output_token = tokenizer(nf)
        yield{'text':nf}
        count += 1

In [ ]:
def generate_validation():
    data = load_dataset("ananyarn/Algorithm_and_Python_Source_Code", streaming=True, split="train", download_mode="force_redownload")
    samp_total = 10000
    valid_p = 0.1
    samp_train=int(samp_total*(1-valid_p))
    count = 0
    for sample in iter(data):
        if count < samp_train:
            count+=1
            continue
        if count >= samp_total:
            break
        q = sample['Algorithm']
        start = q.find("1")
        question = q[start:].strip()
        answer = sample['output'].strip()
        nf = f'<s>[INST]{question}[/INST] ```python\n{answer}```</s>'
        output_token = tokenizer(nf)
        yield{'text':nf}
        count += 1

In [ ]:
config_details=GPTQConfig(bits=4, use_exllama=False, tokenizer=tokenizer)

In [ ]:
dev_map = "auto"

In [ ]:
model = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-Instruct-v0.1-GPTQ", quantization_config=config_details, device_map=dev_map, token=True)

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. ['use_cuda_fp16', 'use_exllama', 'max_input_length', 'exllama_config', 'disable_exllama']) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.
CUDA extension not installed.
CUDA extension not installed.


In [ ]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLU()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32000, bias=False)
)


In [ ]:
model.config.use_cache=False

In [ ]:
model.config.pretraining_tp=1

In [ ]:
model.gradient_checkpointing_enable()

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
model.config.window = 256

In [ ]:
peft_config = LoraConfig(lora_alpha=16, lora_dropout=0.1, r=32, bias="none", task_type="CAUSAL_LM", target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj", "lm_head",],)

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
training_arguments = TrainingArguments(output_dir="./get_python", per_device_train_batch_size=4, per_device_eval_batch_size=4, gradient_accumulation_steps=4, optim="paged_adamw_32bit", save_steps=50, max_grad_norm=0.3, weight_decay=0.01, learning_rate=2e-5, lr_scheduler_type="constant", evaluation_strategy="steps", logging_steps=50, num_train_epochs=1, max_steps=250, warmup_ratio=0.03, group_by_length=True, fp16=False, push_to_hub=True)

In [ ]:
train_generator=Dataset.from_generator(generate_train)
val_generator=Dataset.from_generator(generate_validation)

In [ ]:
tokenizer.padding_side="right"

In [ ]:
trainer=SFTTrainer(model=model, train_dataset=train_generator, eval_dataset=val_generator, peft_config=peft_config, dataset_text_field="text", args=training_arguments, tokenizer=tokenizer, packing=False, max_seq_length=512)

In [ ]:
trainer.train()

/home/cs-ai-04/anaconda3/envs/ml/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
50,0.832600,0.704576
100,0.640400,0.608044
150,0.577100,0.570125
200,0.563700,0.566240
250,0.552000,0.571804


/home/cs-ai-04/anaconda3/envs/ml/lib/python3.10/site-packages/peft/utils/save_and_load.py:134: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
/home/cs-ai-04/anaconda3/envs/ml/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/cs-ai-04/anaconda3/envs/ml/lib/python3.10/site-packages/peft/utils/save_and_load.py:134: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Sett

TrainOutput(global_step=250, training_loss=0.6331560745239257, metrics={'train_runtime': 5106.6394, 'train_samples_per_second': 0.783, 'train_steps_per_second': 0.049, 'total_flos': 1918182537658368.0, 'train_loss': 0.6331560745239257, 'epoch': 0.44})

In [ ]:
trainer.push_to_hub()

/home/cs-ai-04/anaconda3/envs/ml/lib/python3.10/site-packages/peft/utils/save_and_load.py:134: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")


CommitInfo(commit_url='https://huggingface.co/ananyarn/get_python/commit/44b7488d0ba7aa2ce64929362428318e00e6b22d', commit_message='End of training', commit_description='', oid='44b7488d0ba7aa2ce64929362428318e00e6b22d', pr_url=None, pr_revision=None, pr_num=None)